In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns',None)

start_date = input("Enter start date in yyyy/mm/dd format:")
start_date = start_date.replace('/','')

end_date = input("Enter end date in yyyy/mm/dd format:")
end_date = end_date.replace('/','')

Enter start date in yyyy/mm/dd format:1983/01/01
Enter end date in yyyy/mm/dd format:2023/06/01


In [2]:
districts = ["Alwal","Banjara Hills","Bandlaguda","Secunderabad","Kukatpalli","Malkajigiri"]

predictions_df = pd.DataFrame(columns=["District", "Maximum Temperature", "Minimum Temperature","Relative Humidity", "Precipitation", "Surface Pressure","Wind Speed at 10M Range"])

for district in districts:
    address = district
    geolocator = Nominatim(user_agent = "Vinay")
    location = geolocator.geocode(address)
    print(location.address)
    print((location.latitude,location.longitude))
    latitude = location.latitude
    longitude = location.longitude
    
    ## Represenation of lat & Long on map
    import folium
    lat, lon = latitude, longitude
    m = folium.Map(location=[lat, lon], zoom_start=11)
    folium.Marker([lat, lon], popup='My Point').add_to(m)
    m
    url= f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M_MAX,T2M_MIN,RH2M,PRECTOTCORR,PS,WS10M_RANGE&community=RE&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=CSV"
    urllib.request.urlretrieve(url,'dataset1.csv')
    
    #Preprocesssing the data
    df= pd.read_csv('dataset1.csv',skiprows=14)
    df['YEAR'] = df.YEAR.astype(str)
    df['MO'] = df.MO.astype(str)
    df['DY'] = df.DY.astype(str)

    df['date']=df['YEAR'].str.cat(df['MO'],sep = '/')
    df['DATE']=df['date'].str.cat(df['DY'],sep = '/')
    
    df.drop(columns=['YEAR','MO','DY','date'],axis=1,inplace = True)
    df.columns
    df.set_index(['DATE'],inplace=True)
    
    condition = df['T2M_MAX'] < 0
    df.drop(df[condition].index, inplace=True)
    df.describe()
    
    # Normalizing the dta using Min Max scaler
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)
    
    # Splitting training and test data 
    from sklearn.model_selection import train_test_split
    train, test = train_test_split(df_scaled, test_size = 0.2, shuffle = False)
    x_train, y_train, x_test, y_test = [] , [] , [] , []
    for i in range (1, len(train)):
        x_train.append(train[i-1])
        y_train.append(train[i])
    for i in range (1, len(test)):
        x_test.append(test [i-1])
        y_test.append(test[i])
    
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_test, y_test = np.array(x_test), np.array(y_test)
    x_train.shape, y_train.shape, x_test.shape, y_test.shape
    
    from keras.models import Sequential
    from keras.layers import Dense, Activation
    from keras.optimizers import Adam
    from keras.models import load_model
    from keras.layers import Dropout
    
    model = Sequential()
    model.add(Dense(6, input_dim= 6 ,activation= 'relu'))
    model.add(Dense(8, activation= 'relu'))
    model.add(Dense(8, activation= 'relu'))
    model.add(Dense(6))
    model .summary()
    model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
    
    history = model.fit(x_train, y_train, validation_data = (x_test,y_test),epochs = 200,batch_size = 15 , shuffle = False)
    
    y_pred = model.predict(x_test)
    y_pred = scaler.inverse_transform(y_pred)
    actual_y_pred = scaler.inverse_transform(y_test)
    
    T = range(y_pred.shape[0])
    
    # Cross validation using kfold
    
    from sklearn.model_selection import KFold
    cv = KFold(n_splits=10, shuffle=False)
    results = []
    for tr, tt in cv.split(x_train,y_train):
        model.fit(x_train[tr], y_train[tr])
        prediction = model.predict(x_train[tt])
        results.append((prediction, tt))
        
    input_data = []
    print('enter the weather parameters of previous day: ')
    attr1 = float(input("enter Maximum Temperature: "))
    attr2 = float(input("Enter minimum Temperature: "))
    attr3 = float(input("enter Relative Humidity:"))
    attr4 = float(input("enter Precipitation: "))
    attr5 = float(input ("Enter surface Pressure: "))
    attr6 = float(input("Enter Wind Speed at 10M Range: "))
        
    input_data.append(attr1)
    input_data.append(attr2)
    input_data.append(attr3)
    input_data.append(attr4)
    input_data.append(attr5)
    input_data.append(attr6)
        
    input_data = np.array(input_data)
    input_data.shape = (1,6)
    print("Input Data: ", input_data)
    input_data = scaler.transform(input_data)
    print('Scaled Input Data:', input_data)

    pred1 = model.predict(input_data)
    pred2 = scaler.inverse_transform(pred1)
    print('predicted Data: ',pred2[0])
        
    predictions_df = predictions_df.append({"District": district,
                                            "Maximum Temperature": pred2[0][0],
                                            "Minimum Temperature": pred2[0][1],
                                            "Relative Humidity": pred2[0][2],
                                            "Precipitation": pred2[0][3],
                                            "Surface Pressure": pred2[0][4],
                                            "Wind Speed at 10M Range": pred2[0][5]}, ignore_index=True)
    
    
 

Alwal, Ward 134 Alwal, Greater Hyderabad Municipal Corporation North Zone, Hyderabad, Alwal mandal, Medchal–Malkajgiri, Telangana, 500010, India
(17.5022292, 78.5088584)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 42        
                                                                 
 dense_1 (Dense)             (None, 8)                 56        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 6)                 54        
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
788/788 

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_7052\1665724543.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({"District": district,


Banjara Hills, Greater Hyderabad Municipal Corporation Central Zone, Hyderabad, Shaikpet mandal, Hyderabad, Telangana, 500034, India
(17.4177464, 78.4399014)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 6)                 42        
                                                                 
 dense_5 (Dense)             (None, 8)                 56        
                                                                 
 dense_6 (Dense)             (None, 8)                 72        
                                                                 
 dense_7 (Dense)             (None, 6)                 54        
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
788/788 [=========

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_7052\1665724543.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({"District": district,


Bandlaguda, Ward 11 Nagole, Greater Hyderabad Municipal Corporation East Zone, Hyderabad, Uppal mandal, Medchal–Malkajgiri, Telangana, 500068, India
(17.3714879, 78.5727265)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 6)                 42        
                                                                 
 dense_9 (Dense)             (None, 8)                 56        
                                                                 
 dense_10 (Dense)            (None, 8)                 72        
                                                                 
 dense_11 (Dense)            (None, 6)                 54        
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
78

...

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_7052\1665724543.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({"District": district,


Reliance Digital Kukatpalli, Kukkatpally Main Road - Mumbai Highway, Sangeet Nagar, Ward 115 Balaji Nagar, Hyderabad, Kukatpally mandal, Medchal–Malkajgiri, Telangana, 500072, India
(17.4820942, 78.414501)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 6)                 42        
                                                                 
 dense_17 (Dense)            (None, 8)                 56        
                                                                 
 dense_18 (Dense)            (None, 8)                 72        
                                                                 
 dense_19 (Dense)            (None, 6)                 54        
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
________________________________________________

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_7052\1665724543.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({"District": district,


SubCentre, Malkajigiri (Urban), 1st Lane, R.K. Nagar, Ward 140 Malkajgiri, Malkajgiri, Hyderabad, Malkajgiri mandal, Medchal–Malkajgiri, Telangana, 500047, India
(17.4507261, 78.5352355)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 6)                 42        
                                                                 
 dense_21 (Dense)            (None, 8)                 56        
                                                                 
 dense_22 (Dense)            (None, 8)                 72        
                                                                 
 dense_23 (Dense)            (None, 6)                 54        
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________
E

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_7052\1665724543.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({"District": district,


In [3]:
predictions_df.head()

,District,Maximum Temperature,Minimum Temperature,Relative Humidity,Precipitation,Surface Pressure,Wind Speed at 10M Range
0,Alwal,41.831860,28.920076,31.743231,1.784926,94.808357,6.066323
1,Banjara Hills,41.309570,28.765596,34.526794,1.111846,94.842369,6.079610
2,Bandlaguda,41.679264,28.658834,32.954350,1.361062,94.808693,6.328476
3,Secunderabad,41.678566,28.221781,33.593113,1.191362,94.715004,5.785617
4,Kukatpalli,41.688240,28.179619,32.127789,2.924545,93.989189,5.284229


In [4]:
predictions_df.to_excel("Final_hyd.xlsx", index=False)